In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import datetime as dt
from splinter import Browser
import time
import re


In [3]:
def start_browser():
    # browser init for webscraping
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    return browser

In [47]:
# setting date parameters
start_date = dt.date(2003, 3,17)
format_change_date1 =dt.date(2003,4,10)
format_change_date2 =dt.date(2003,4,24)
end_date = dt.date(2003, 7,12)
cur_date = start_date
# Setting browers params
output_df=pd.DataFrame({'Date':'', 'Country':'','Cases':'','Deaths':''},index=[])
home ='https://www.who.int/csr/sars/country/en/'
browser=start_browser()
browser.visit(home)
# loop though WHO links
while cur_date!= end_date:
    # search string formatting
    date_string = cur_date.strftime("%d %B %Y")
    if date_string[0]=='0':
        date_string=date_string[1:]
    # Catch some missing dates
    try:
        browser.click_link_by_partial_text(date_string)
        html = bs(browser.html, 'html.parser')
        tab_tag=html.find('table')
        # read html table into df
        temp_df = pd.read_html(str(tab_tag))[0]
        browser.back()
        # Deal with some region name formatting
        if cur_date == dt.date(2003,6,9):
            temp_df = temp_df.iloc[:-1]
        for index, row in temp_df.iterrows():
            if row[0].find('Hong Kong') != -1:
                temp_df.iloc[index,0]="Hong Kong"
            elif row[0].find('Taiwan') != -1:
                temp_df.iloc[index,0]="Taiwan"
            elif row[0].find('Guangdong') != -1:
                temp_df.iloc[index,0]="China"
            elif row[0].find('Macao') != -1:
                temp_df.iloc[index,0]="China"
            elif row[0].find('China +') != -1:
                temp_df.iloc[index,0]="China"
        # table format 1
        if cur_date < format_change_date1:
            temp_df=temp_df.iloc[1:,:3]
            temp_df.rename(columns={0:'Country',1:'Cases',2:'Deaths'},inplace=True)
            temp_df['Date']=cur_date
            temp_df=temp_df[['Date','Country','Cases','Deaths']]
            output_df=output_df.append(temp_df)
        # table format 2
        elif cur_date < format_change_date2:
            temp_df=temp_df.iloc[2:,:4]
            temp_df.rename(columns={0:'Country',1:'Cases',3:'Deaths'},inplace=True)
            temp_df['Date']=cur_date
            temp_df=temp_df[['Date','Country','Cases','Deaths']]
            output_df=output_df.append(temp_df)
        # format 3
        else:
            append_df=pd.DataFrame()
            append_df['Country']=temp_df.iloc[:,0]
            append_df['Cases']=temp_df.iloc[:,1]
            append_df['Deaths']=temp_df.iloc[:,3]
            append_df['Date']=cur_date
            append_df=append_df[['Date','Country','Cases','Deaths']]
            output_df=output_df.append(append_df)
        print(cur_date)
        cur_date += dt.timedelta(days=1)
    except:
        print('no data for '+date_string)
        cur_date += dt.timedelta(days=1)
browser.quit()
output_df

2003-03-17
2003-03-18
2003-03-19
2003-03-20
2003-03-21
2003-03-22
no data for 23 March 2003
2003-03-24
2003-03-25
2003-03-26
2003-03-27
2003-03-28
2003-03-29
no data for 30 March 2003
2003-03-31
2003-04-01
2003-04-02
2003-04-03
2003-04-04
2003-04-05
2003-04-06
2003-04-07
2003-04-08
2003-04-09
2003-04-10
2003-04-11
2003-04-12
no data for 13 April 2003
2003-04-14
2003-04-15
2003-04-16
2003-04-17
2003-04-18
2003-04-19
no data for 20 April 2003
2003-04-21
2003-04-22
2003-04-23
2003-04-24
2003-04-25
2003-04-26
no data for 27 April 2003
2003-04-28
2003-04-29
2003-04-30
2003-05-01
2003-05-02
2003-05-03
2003-05-04
2003-05-05
2003-05-06
2003-05-07
2003-05-08
2003-05-09
2003-05-10
no data for 11 May 2003
2003-05-12
2003-05-13
2003-05-14
2003-05-15
2003-05-16
2003-05-17
no data for 18 May 2003
2003-05-19
2003-05-20
2003-05-21
2003-05-22
2003-05-23
2003-05-24
no data for 25 May 2003
2003-05-26
2003-05-27
2003-05-28
2003-05-29
2003-05-30
2003-05-31
2003-06-01
2003-06-02
2003-06-03
2003-06-04
2003-0

,Date,Country,Cases,Deaths
1,2003-03-17,Germany,1,0
2,2003-03-17,Canada,8,2
3,2003-03-17,Singapore,20,0
4,2003-03-17,Hong Kong,95,1**
5,2003-03-17,Switzerland,2,0
...,...,...,...,...
28,2003-07-11,Thailand,9,2
29,2003-07-11,United Kingdom,4,0
30,2003-07-11,United States^7,75,0
31,2003-07-11,Viet Nam,63,5


In [57]:
output_df['Country'].unique()

array(['Germany', 'Canada', 'Singapore', 'Hong Kong', 'Switzerland',
       'Thailand', 'Viet Nam', 'Total', 'China', 'Taiwan', 'Slovenia',
       'United Kingdom', 'China+', 'Spain', 'United States', 'Italy',
       'Republic of Ireland', 'China\xa0+', 'France', 'Romania', 'China3',
       'Australia', 'Belgium', 'Brazil', 'Malaysia', 'Country',
       'Indonesia', 'Japan', 'Kuwait', 'Philippines', 'South Africa',
       'Sweden', 'Viet Nam 5', 'India', 'Mongolia', 'United States5',
       'Bulgaria', 'China^5', 'Republic of Korea', 'Poland',
       'New Zealand', 'Colombia', 'Finland', 'Canada^4',
       'Russian Federation', 'Philippines^5', 'Philippines^7',
       'United States^8', 'United States^7'], dtype=object)

In [5]:
# output_df_cleaned = output_df
alphabet_only = re.compile('[^a-zA-Z ]')
test ='U S A^7'
alphabet_only.sub('',test)
#First parameter is the replacement, second parameter is your input string
# output_df_cleaned['Country']=output_df_cleaned['Country'].apply(lambda x:alphabet_only.sub('',x))
# output_df_cleaned['Country'].apply(lambda x:alphabet_only.sub('',x))
# for index, row in output_df.iterrows():
#     country_string = row['Country']
#     if country_string.find('United States') !=-1:
#         output_df_cleaned.iloc[index]['Country']='United States'
#     elif country_string.find('Philippines') !=-1:
#         output_df_cleaned.iloc[index]['Country']='Philippines'
#     elif country_string.find('China') !=-1:
#         output_df_cleaned.iloc[index]['Country']='China'
#     elif country_string.find('Viet Nam') !=-1:
#         output_df_cleaned.iloc[index]['Country']='Viet Nam'

'U S A'

In [59]:
output_df_cleaned['Country'].unique()

array(['Germany', 'Canada', 'Singapore', 'Hong Kong', 'Switzerland',
       'Thailand', 'Viet Nam', 'Total', 'China', 'Taiwan', 'Slovenia',
       'United Kingdom', 'China+', 'Spain', 'United States', 'Italy',
       'Republic of Ireland', 'China\xa0+', 'France', 'Romania', 'China3',
       'Australia', 'Belgium', 'Brazil', 'Malaysia', 'Country',
       'Indonesia', 'Japan', 'Kuwait', 'Philippines', 'South Africa',
       'Sweden', 'Viet Nam 5', 'India', 'Mongolia', 'United States5',
       'Bulgaria', 'China^5', 'Republic of Korea', 'Poland',
       'New Zealand', 'Colombia', 'Finland', 'Canada^4',
       'Russian Federation', 'Philippines^5', 'Philippines^7',
       'United States^8', 'United States^7'], dtype=object)

In [37]:
output_df_cleaned

,Date,Country,Cases,Deaths
1,2003-03-17,Germany,1,0
2,2003-03-17,Canada,8,2
3,2003-03-17,Singapore,20,0
4,2003-03-17,HongKong,95,1**
5,2003-03-17,Switzerland,2,0
...,...,...,...,...
28,2003-07-11,Thailand,9,2
29,2003-07-11,UnitedKingdom,4,0
30,2003-07-11,UnitedStates,75,0
31,2003-07-11,VietNam,63,5
